In [1]:
with open('env-final.fasta.nwk', 'r') as file_handle:
    tree = file_handle.read()
coors = []

for i in range(len(tree)):    
    if (tree[i] == '(' and tree[i+1] == 'N') or (tree[i] == ',' and tree[i+1] == 'N'):
        extraction = ''
        #add the extraction
        
        for add in range(i+1, i+50):
            if tree[add] == '_' and add != i+3:
                break
            else:
                extraction += tree[add]
        if not extraction[:7] == 'NC_0000':      
            for j in range(len(extraction)):
                if extraction[j] == '.':
                    accession = extraction[:j+2]
                    startend = extraction[j+2:]
                    if startend[0] == 'c':
                        start = int(startend.split('-')[0][1:]) - 10000
                        end = int(startend.split('-')[1]) + 10000
                    else:
                        start = int(startend.split('-')[0]) - 10000
                        end = int(startend.split('-')[1]) + 10000
                    coors.append([accession, start, end])

In [7]:
import os
import subprocess
for seq in coors:
    if seq[1] > 0:
        subprocess.run('efetch -db nucleotide -id {} -seq_start {} -seq_stop {} -format fasta > {}.fasta'.format(seq[0], seq[1], seq[2], seq[0]), shell=True)

In [1]:
#extract sequences from NCBI BLAST hit table
#filter sequences by e-value & best hit (unique sequence)
import pandas as pd
from Bio import SeqIO
df = pd.read_csv('env_hit.csv')
print(df.head())

#e-value cutoff 1e-10

def extract_coord(i, df):
    accession = str(df.loc[i, 'Subject'])
    start = str(df.loc[i, 'Subject_start'])
    end = str(df.loc[i, 'Subject_end'])
    return accession, start, end

grandlist = []
for i in range(len(df.index)):
        value = str(df.loc[i, 'E-value'])
        if value == '0':
            accession, start, end = extract_coord(i, df)
            grandlist.append([accession, int(start)-10000, int(end)+10000])
        elif 'e' in value[-5:]:
            if value[-3] == '1' or value[-3] == '2':
                accession, start, end = extract_coord(i, df)
                grandlist.append([accession, int(start)-10000, int(end)+10000])
            elif not value[-2] == '0':
                accession, start, end = extract_coord(i, df)
                grandlist.append([accession, int(start)-10000, int(end)+10000])

print(len(grandlist))
print(grandlist[0])

#fetch the sequences with efetch
import os
import subprocess
i = 0
for seq in grandlist:
    if seq[1] > 0:
        subprocess.run('efetch -db nucleotide -id {} -seq_start {} -seq_stop {} -format fasta > {}_{}.fasta'.format(seq[0], seq[1], seq[2], seq[0], seq[1]), shell=True)
    i += 1
    if i % 50 == 0:
        print(i)

     Query         Subject  Identity  Length  Mismatch  Gap_opens  \
0  unnamed  NW_024342471.1    94.059     303        18          0   
1  unnamed  NW_024342471.1    58.766     308       111          7   
2  unnamed  NW_024342471.1    72.727     154        37          4   
3  unnamed  NW_024342471.1    65.517     116        37          1   
4  unnamed  NW_024342471.1    42.727     220        96          5   

   Query_start  Query_end  Subject_start  Subject_end        E-value  \
0            1        303         435931       436839   0.000000e+00   
1            1        302       14976348     14977241  2.700000e-100   
2          151        301       12133025     12133480   9.240000e-93   
3            1        116       12132585     12132923   9.240000e-93   
4           98        302        3554502      3553888   3.490000e-38   

   bit_score  Positives  Q_frame  S_frame  
0      592.0      96.04        0        1  
1      329.0      69.48        0        3  
2      209.0      81

In [2]:
#merge all the files
import os
from Bio import SeqIO
dir = os.getcwd()
filelist = []
for file in os.listdir(dir):
    filename = os.fsdecode(file)
    if filename.endswith(".fasta"):
        filelist.append(filename)
        continue
    else: 
        continue
print(len(filelist))
i = 0

for file in filelist:
    for seq_record in SeqIO.parse('{}'.format(file), 'fasta'):
        with open('combined.fasta', 'a') as output_handle:
            output_handle.write('>' + str(seq_record.description) + '\n' + str(seq_record.seq) + '\n')
        i += 1
        if i % 10 == 0:
            print(i)

322
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
